In [1]:
import numpy as np
import pandas as pd
import redis


In [2]:
# connect to redis client
# redis- 
hostname = 
portnumber = 
password = 

r = redis.StrictRedis(host=hostname,
                    port=portnumber,
                    password=password)




In [3]:
r.ping()

True

In [6]:
# r.keys()

[b'mylist', b'fool', b'KEY', b'MESSAGE']

## Load the data

In [4]:
np_file = np.load('dataframe_students_teachers.npz',allow_pickle=True)
x_values = np_file['arr_0']
col_names = np_file['arr_1']

df = pd.DataFrame(x_values,columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."


In [5]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967...",Scarlett Johansson@Student


In [6]:
records = df[['name_role','Facial_Features']].to_dict(orient='records')
len(records)

5

### save data in redis

In [7]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']


    # convert numpy array into bytes
    vector_bytes = vector.tobytes()
    r.hset(name = 'academy:register',key = name_role,value=vector_bytes)

### Retrieve Records from redis

In [8]:
retrieve_dict = r.hgetall(name='academy:register')
retrieve_series = pd.Series(retrieve_dict)



In [9]:
retrieve_series

b'Angelina Jolie@Student'        b'3f\xdd>\xae\x94\x89?\xeb\xb5\x03\xbeN\xa9\xe...
b'Chris Evans@Student'           b')]8?\x05\xf5\xbb\xbf83G\xbf\xcc\xe4\xba\xbf\...
b'Morgan Freeman@Teacher'        b'\xee\x7f\xf8>\x87\x87\x1e\xbfja\xd0\xbe\xbe\...
b'Aviral Dwivedi@Students'       b'\xcc\xf7\xfe\xbe\xa09\xe6?\xef\xac[\xbf\x06\...
b'Scarlett Johansson@Student'    b'K\xe9C>\x8d\x14\xfa\xbe\x8eY\xd8\xbf\xd3\x81...
b'Barack Obama@Teacher'          b'5\xd6r?\xa5\xf3\x0e\xbf]1\xc2=\xde\xcb\xa3\x...
b'Akshat@Student'                b'\xdd\x88i\xbf\xb8\x00\x9f>\xef\xd1\xf7<A\xf9...
b'Aviral@Student'                b'Lov\xbd:\x8cS\xbe\xe5\xa4s\xbf\x97\x9a\x7f\x...
dtype: object

In [10]:
# np.frombuffer(retrieve_series[b'Morgan Freeman@Teacher'],dtype=np.float(32).shape)
retrieve_series = retrieve_series.apply(lambda x : np.frombuffer(x,dtype=np.float32))
index = retrieve_series.index
index = list(map(lambda x : x.decode(),index))

In [11]:
retrieve_series.index = index

In [14]:
retrieve_df = retrieve_series.to_frame().reset_index()
retrieve_df.columns = ['name_role','facial_features']
retrieve_df

,name_role,facial_features
0,Angelina Jolie@Student,"[0.43242034, 1.0748498, -0.12862365, -1.833291..."
1,Chris Evans@Student,"[0.7201715, -1.4684149, -0.7781253, -1.4601073..."
2,Morgan Freeman@Teacher,"[0.48535103, -0.6192555, -0.4069932, 1.0687788..."
3,Aviral Dwivedi@Students,"[-0.4979843, 1.7986336, -0.8581075, 0.22268304..."
4,Scarlett Johansson@Student,"[0.19131963, -0.48843804, -1.690233, 0.4423967..."
5,Barack Obama@Teacher,"[0.94858104, -0.5584052, 0.094820715, -1.27965..."
6,Akshat@Student,"[-0.9122446, 0.31055236, 0.030251471, 0.167943..."
7,Aviral@Student,"[-0.060164735, -0.20658961, -0.95173484, -0.99..."


### Registration Form

In [13]:
import cv2
from insightface.app import FaceAnalysis

In [15]:
# configure face analysis
faceapp = FaceAnalysis(name = 'buffalo_sc',
                    root='insightface_model',
                    providers = ['CPUExecutionProvider'])

faceapp.prepare(ctx_id = 0 , det_size=(640,640),det_thresh=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: insightface_model\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


#### Step : 1 Collect Person Name and Role

In [16]:
person_name = input("Enter your Name:")
trials = 3
for i in range (trials):
    role = input('''Please Choose
                1. Student
                2. Teacher
                
                Enter number either 1 or 2''')
    if role in ('1','2'):
        if role == '1':
            role='Student'
        else:
            role = 'Teacher'
        break
    else:
        print("Invalid entry")
        if i==3:
            print("Exceeds maximum trials")
key = person_name+'@'+role
print("Your Name:",person_name)
print("Your Role:",role)
print("Key:",key)

Your Name: Deepak
Your Role: Student
Key: Deepak@Student


#### Step : 2 Collect Samples

In [18]:
cap = cv2.VideoCapture(0) # 0 default camera , 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print("unable to read camera")
        break
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1,y1,x2,y2 = res['bbox'].astype(int)
        cv2.rectangle(frame,(x1,y1),(x2,y2),(0,255,0),1)

        # facial features
        embeddings = res['embedding']
        face_embeddings.append(embeddings)
    if sample >=200:
        break
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) == ord('q'): # this will trigger when you press q(lowercase) in keyboard.
        break
cap.release()
cv2.destroyAllWindows()

In [19]:
len(face_embeddings)

200

#### optimize data and save in redis databse

In [20]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [21]:
# convert into bytes
x_mean_bytes = x_mean.tobytes()

In [22]:
# save key and facial bytes into redis database
r.hset(name = 'academy:register',key = key,value=x_mean_bytes)

1